In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.utils import shuffle
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook as tqdm
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D
from keras import regularizers
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [64]:
%%time
data = pd.read_json("data.json")

CPU times: user 6.37 s, sys: 2.15 s, total: 8.52 s
Wall time: 8.62 s


In [65]:
X = data['query']
y = data['tags']

In [66]:
data = pd.DataFrame({'query':X, 'tags': y})
#data = shuffle(data, random_state=5)

In [67]:
data = data.head(15000)

In [93]:
data.to_json("data1.json")

In [68]:
X = data['query']
y = data['tags']

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0, random_state=5)

In [70]:
X_test

Series([], Name: query, dtype: object)

In [71]:
def padd_data(data, max_len=29, value=0, mode='post'):
    return pad_sequences(data, maxlen=max_len, value=value, padding=mode, dtype='str')

In [72]:
%%time
X_train = padd_data(X_train, max_len=29, value=0)
X_test = padd_data(X_test, max_len=29, value=0)

CPU times: user 232 ms, sys: 14.2 ms, total: 246 ms
Wall time: 242 ms


In [73]:
%%time
y_train = padd_data(y_train, max_len=29, value=0)
y_test = padd_data(y_test, max_len=29, value=0)
y_train[y_train == '0.0'] = 'stop_word'
y_test[y_test == '0.0'] = 'stop_word'

CPU times: user 255 ms, sys: 31.1 ms, total: 286 ms
Wall time: 289 ms


In [74]:
all_tags = list(set(y_train.reshape((y_train.shape[0] * y_train.shape[1]))))
print (all_tags)

['B-filter', 'B-predefined_filter', 'B-filter_value', 'B-dimension', 'stop_word', 'I-predefined_filter', 'I-filter', 'B-metric', 'I-dimension', 'I-metric']


In [75]:
%%time
a = list(set(X_train.reshape((X_train.shape[0] * X_train.shape[1]))))
b = list(set(X_test.reshape((X_test.shape[0] * X_test.shape[1]))))
a = list(set(a + b))

CPU times: user 184 ms, sys: 1.37 ms, total: 185 ms
Wall time: 185 ms


In [76]:
word2numb = dict(zip(sorted(a), list(range(len(a)))))
numb2word = dict(zip(list(range(len(a))), sorted(a)))
tag2numb = dict(zip(sorted(all_tags), list(range(len(all_tags)))))
numb2tag = dict(zip(list(range(len(all_tags))), sorted(all_tags)))

In [77]:
n_classes = len(all_tags)
n_vocab = len(word2numb)
print(n_classes)
print(n_vocab)

10
310


In [78]:
%%time
X_train_numb = np.array([list(map(lambda x: word2numb[x], w)) for w in X_train])
X_test_numb = np.array([list(map(lambda x: word2numb[x], w)) for w in X_test])
y_train_numb = np.array([list(map(lambda x: tag2numb[x], y)) for y in y_train])
y_test_numb = np.array([list(map(lambda x: tag2numb[x], y)) for y in y_test])

CPU times: user 652 ms, sys: 6.97 ms, total: 659 ms
Wall time: 659 ms


In [79]:
def get_accuracy(y_true, y_pred):
    #print (y_true)
    #print (y_pred)
    assert y_true.shape == y_pred.shape 
    return sum(list(int(np.all(y_true[i] == y_pred[i])) for i in range(y_pred.shape[0]))) / len(y_true)

In [80]:
def prediction(model, x=X_test_numb):
    labels_pred = []

    for n_batch, sent in tqdm(enumerate(x)):
        pred = model.predict_on_batch(sent[np.newaxis,:])
        labels_pred.append(np.argmax(pred,-1)[0])

    labels_pred = [ list(map(lambda x: numb2tag[x], y)) for y in labels_pred]
    return labels_pred

In [81]:
def training(n_epochs = 2):
    for i in tqdm(range(n_epochs)):
        print("Epoch {}".format(i))

        for n_batch, sent in tqdm(enumerate(X_train_numb)):
            # Make labels one hot
            label = np.eye(len(all_tags))[y_train_numb[n_batch]][np.newaxis,:] 
            # View each sentence as a batch
            sent = sent[np.newaxis,:]
            if sent.shape[1] > 1: #ignore 1 word sentences
                model.train_on_batch(sent, label)
        print ("Train accuracy: ", get_accuracy(np.array(prediction(model, X_train_numb)), np.array(y_train)))
        print ("Validation accuracy: ", get_accuracy(np.array(prediction(model, X_test_numb)), np.array(y_test)))

In [82]:
model = Sequential()
model.add(Embedding(n_vocab, 100))
model.add(Convolution1D(128, 4, padding='same', activation='relu'))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')

In [83]:
training(1)

Widget Javascript not detected.  It may not be installed or enabled properly.


Epoch 0


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Train accuracy:  0.9804666666666667


Widget Javascript not detected.  It may not be installed or enabled properly.


AssertionError: 

In [22]:
print ("Train accuracy: ", get_accuracy(np.array(prediction(model, X_train_numb)), np.array(y_train)))


Train accuracy:  0.986


In [84]:
model.save_weights("best_weights")

In [86]:
model1 = Sequential()
model1.add(Embedding(n_vocab, 100))
model1.add(Convolution1D(128, 4, padding='same', activation='relu'))
model1.add(Bidirectional(LSTM(100,return_sequences=True)))
model1.add(Dropout(0.5))
model1.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model1.compile('rmsprop', 'categorical_crossentropy')

In [87]:
model1.load_weights("best_weights")

In [88]:
def predict_one_query(query, model):
    query = padd_data([query.split()], value=0)[0]
    query = np.array(list(map(lambda x: word2numb[x], query)))
    pred = model.predict_on_batch(query[np.newaxis,:])
    pred = np.argmax(pred,-1)[0]
    return list(map(lambda x: numb2tag[x], pred))

In [89]:
s = "new users by source for country less germany"

In [90]:
ss = np.array(s.split())
t = list(map(lambda x: word2numb[x], ss))

In [91]:
t

[176, 293, 52, 256, 115, 75, 151, 119]

In [92]:
%%time
print (predict_one_query(s, model=model1))

['B-metric', 'I-metric', 'stop_word', 'B-dimension', 'stop_word', 'B-filter', 'stop_word', 'B-filter_value', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word', 'stop_word']
CPU times: user 394 ms, sys: 227 ms, total: 622 ms
Wall time: 606 ms


#### С этого места фичи из CRF NER

In [ ]:
from nlu.cfgrammar.GADictionaryGroup import *
from nlu.cfgrammar.GrammarBuilder import *
from nlu.cfgrammar.DictionaryGroup import *
from nlu.cfgrammar.DictionaryGroupsFetcher import *
import random
import nltk
import itertools

In [135]:
random.seed(5)

In [136]:
ga_groups = DictionaryGroupsFetcher.fetch_dictionary_groups(integration_ids=[20431])
metric_dictionary = ga_groups[0][0].original_metric_dictionary
dimension_dictionary = ga_groups[0][0].original_dimension_dictionary
filter_dictionary = ga_groups[0][0].original_filter_dictionary
predefined_filter_dictionary = ga_groups[0][0].original_predefined_filter_dictionary

In [139]:
class DatasetGenerator:
    def __init__(self, dictionaries={}, stop_words={}, filter_values=[]):
        self.dictionaries = dictionaries
        self.stop_words = stop_words
        self.filter_values = filter_values
        self.dic_word_frequency = {}

    def split_names_in_dictionaries(self):
        for dictionary_name in self.dictionaries:
            for i in range(len(self.dictionaries[dictionary_name])):
                for j in range(len(self.dictionaries[dictionary_name][i][1])):
                    try:
                        self.dictionaries[dictionary_name][i][1][j] = self.dictionaries[dictionary_name][i][1][j].split()
                    except:
                        self.dictionaries[dictionary_name][i][1][j] = []

    def number_of_uniq_words_in_dictionary(self, splitted_dictionary):
        return len(set([token for name in splitted_dictionary for syn in name[1] for token in syn]))

    def add_stop_word(self):
        return self.stop_words['stop_word'][rndint(0, len(self.stop_words['stop_word']))]
    
    
    def precalculation(self, word, splitted_dictionary):
        return set([(ind1, ind2) for ind1, name in enumerate(splitted_dictionary) 
                                for ind2, syn in enumerate(name[1]) if word in syn])
    
    def get_all_words(self, splitted_dictionary):
        return list(set([token for name in splitted_dictionary for syn in name[1] for token in syn]))
    
    def construct_dictionary_frequency(self, big_dictionary):
        list_of_all_words = []
        for name_dic in big_dictionary:
            list_of_all_words += self.get_all_words(big_dictionary[name_dic])
        for word in list_of_all_words:
            self.dic_word_frequency[word] = {}
            for name_dic in big_dictionary:
                self.dic_word_frequency[word][name_dic] = self.precalculation(word, big_dictionary[name_dic])
                
    def findsubsets(self, S, m):
        return list(set(itertools.combinations(S, m)))
    
    def generate_all_names(self, dictionary):
        return[self.findsubsets(name, i)[0] for name in dictionary for i in range(1, len(name)+1)]
            
    def generate_rules(self, position, query, structure, filling):
        if position == -1:
            self.generate_rules(position + 1, query, structure, filling)
            self.generate_rules(position + 1, query + 'show ', structure + ['sw'], filling)
            self.generate_rules(position + 1, query + 'show me ', structure + ['sw', 'sw'], filling)
            #self.generate_rules(position + 1, query + 'show me please ', structure + ['sw', 'sw', 'sw'], filling)
        if position == 0:
            self.generate_rules(position + 1, query + '{m1}', structure + ['m1'], filling+1)
            self.generate_rules(position + 1, query + '{m1} and {m2}', structure + ['m1', 'sw', 'm2'], filling+2)
            #self.generate_rules(position + 1, query + '{m1} {m2}', structure + ['m1', 'm2'], filling+1)
            self.generate_rules(position + 1, query + '{m1} vs {m2}', structure + ['m1', 'sw', 'm2'], filling+2)
            self.generate_rules(position + 3, query + '{pf1} {m1}', structure + ['pf1', 'm1'], filling+1)
        elif position == 1:
            self.generate_rules(position + 1, query, structure, filling)
            self.generate_rules(position + 1, query + ' {d1}', structure + ['d1'], filling+1)
            self.generate_rules(position + 1, query + ' by {d1}', structure + ['sw', 'd1'], filling+1)
            self.generate_rules(position + 1, query + ' for {d1}', structure + ['sw', 'd1'], filling+1)
            self.generate_rules(position + 1, query + ' by {d1} and {d2}', structure + ['sw', 'd1', 'sw', 'd2'], filling+2)
            #self.generate_rules(position + 1, query + ' by {d1} and by {d2}',
            #                    structure + ['sw', 'd1', 'sw', 'sw', 'd2'], filling+2)
        elif position == 2:
            if filling == 3:
                filling = 10
            self.generate_rules(position + 1, query, structure, filling)
            self.generate_rules(position + 1, query + ' {pf1}', structure + ['pf1'], filling+1)
            self.generate_rules(position + 1, query + ' for {pf1}', structure + ['sw', 'pf1'], filling+1)
            self.generate_rules(position + 1, query + ' where {pf1}', structure + ['sw', 'pf1'], filling+1)
        elif position == 3:
            if filling == 3:
                filling = 10
            self.generate_rules(position + 1, query, structure, filling)
            self.generate_rules(position + 1, query + ' {f1}', structure + ['f1'], filling+1)
            self.generate_rules(position + 1, query + ' for {f1}', structure + ['sw', 'f1'], filling+1)
            self.generate_rules(position + 1, query + ' for {f1} and {f2}', structure + ['sw', 'f1', 'sw', 'f2'], filling+2)
            self.generate_rules(position + 1, query + ' for {f1} and for {f2}', structure + ['sw', 'f1', 'sw', 'sw', 'f2'], filling+2)
            self.generate_rules(position + 2, query + ' {fv1} {f1}', structure + ['fv1', 'f1'], filling+2)
        elif position == 4 and 'f1' in structure[-1]:
            if filling == 3:
                filling = 10
            self.generate_rules(position + 1, query + ' contains {fv1}', structure + ['sw', 'fv1'], filling+1)
            self.generate_rules(position + 1, query + ' less {fv1}', structure + ['sw', 'fv1'], filling+1)
            self.generate_rules(position + 1, query + ' greater {fv1}', structure + ['sw', 'fv1'], filling+1)
            #self.generate_rules(position + 1, query + ' greater than {fv1}', structure + ['sw', 'sw', 'fv1'], filling+1)
            #self.generate_rules(position + 1, query + ' equal {fv1}', structure + ['sw', 'fv1'], filling+1)
        else:
            self.rules.append([query, structure])
        
    def generate_queries_denis(self):
        
        '''
        self.metrics = [token for metric in self.dictionaries['metric_dictionary'] for token in metric[1]]
        self.dimensions = [token for dimension in self.dictionaries['dimension_dictionary'] for token in dimension[1]]
        self.filters = [token for filt in self.dictionaries['filter_dictionary'] for token in filt[1]]
        self.predefined_filters = [token for predefined_filter in self.dictionaries['predefined_filter_dictionary'] for
                                   token in predefined_filter[1]]
        '''
        self.split_names_in_dictionaries()
        
        self.metrics = [token for metric in self.dictionaries['metric_dictionary'] for token in metric[1]]
        self.dimensions = [token for dimension in self.dictionaries['dimension_dictionary'] for token in dimension[1]]
        self.filters = [token for filt in self.dictionaries['filter_dictionary'] for token in filt[1]]
        self.predefined_filters = [token for predefined_filter in self.dictionaries['predefined_filter_dictionary'] for
                                   token in predefined_filter[1]]

        self.construct_dictionary_frequency({'metric': self.dictionaries['metric_dictionary'],
                                             'dimension': self.dictionaries['dimension_dictionary'],
                                             'filter': self.dictionaries['filter_dictionary'],
                                             'predefined_filter': self.dictionaries['predefined_filter_dictionary']})
        
        self.total_numbers = np.array(
            [self.number_of_uniq_words_in_dictionary(self.dictionaries[d_name]) for d_name in self.dictionaries])

        self.rules = []
        self.generate_rules(-1, "", [], 0)

        self.queries, self.queries_with_tags, self.labels = [], [], []
        
        '''
        print(self.metrics)
        self.metrics = [['page', 'view', 'per', 'session'], ['avg', 'session', 'duration'], ['new', 'user'], ['conversions', 'rate'], ['signup'], ['users'], ['signup']]
        self.dimensions = [['social', 'activity', 'network', 'action'], ['query', 'product', 'name'], ['social', 'type'], ['browser', 'version'], ['product', 'name'], ['source']]
        self.predefined_filters = [['tablet', 'and', 'desktop', 'traffic'], ['mobile', 'and', 'tablet'], ['new', 'users'], ['ios'], ['tablet', 'traffic']]
        self.filters = [['social', 'activity', 'network', 'action'], ['internal', 'promotion', 'name'], ['device', 'category'], ['app'], ['user', 'gender'], ['network', 'location'], ['app', 'name'] ]
        print(self.metrics)
        '''
        self.metrics_list = self.generate_all_names(self.metrics)
        self.dimensions_list = self.generate_all_names(self.dimensions)
        self.predefined_filters_list = self.generate_all_names(self.predefined_filters)
        self.filters_list = self.generate_all_names(self.filters)

        self.cartmetric1 = cartesian([self.metrics_list, self.metrics_list])
        self.cartdimension1 = cartesian([self.dimensions_list, self.dimensions_list])
        self.cartfilter1 = cartesian([self.filters_list, self.filters_list])
        self.cartpredfilter1 = cartesian([self.predefined_filters_list, self.predefined_filters_list])
                        
        ind = -1
        for rule in tqdm(self.rules):
            ind += 1
            numbers1 = [('m1' in rule[1]) * len(self.metrics_list) + ('m1' not in rule[1]),
                        ('d1' in rule[1]) * len(self.dimensions_list) + ('d1' not in rule[1]),
                        ('f1' in rule[1]) * len(self.filters_list) + ('f1' not in rule[1]),
                        ('pf1' in rule[1]) * len(self.predefined_filters_list) + ('pf1' not in rule[1]),
                        ('fv1' in rule[1]) * 2 + ('fv1' not in rule[1])]
            
            if ind % 4 == 0:
                np.random.shuffle(self.cartmetric1)
                np.random.shuffle(self.cartdimension1)
                np.random.shuffle(self.cartfilter1)
                np.random.shuffle(self.cartpredfilter1)
            
            if 'm2' in rule[1]:
                numbers1[0] *= len(self.metrics_list)
            if 'd2' in rule[1]:
                numbers1[1] *= len(self.dimensions_list)
            if 'f2' in rule[1]:
                numbers1[2] *= len(self.filters_list)
            if 'pf2' in rule[1]:
                numbers1[3] *= len(self.predefined_filters_list)
                
            
            self.cartmetric = self.cartmetric1[:2]
            self.cartdimension = self.cartdimension1[:2]
            self.cartfilter = self.cartfilter1[:2]
            self.cartpredfilter = self.cartpredfilter1[:2]
            
            filter_values = np.array(self.filter_values)[rndint(0, len(self.filter_values),
                                                                numbers1[4])]
            for met1, met2 in self.cartmetric:
                for dim1, dim2 in self.cartdimension:
                    for fil1, fil2 in self.cartfilter:
                        for pfil1, pfil2 in self.cartpredfilter:
                            for filv1 in self.filter_values:
                                label = []
                                for tag in rule[1]:
                                    if tag == 'm1':
                                        for i in range(len(met1)):
                                            if i == 0:
                                                label.append('B-metric')
                                            else:
                                                label.append('I-metric')

                                    if tag == 'm2':
                                        for i in range(len(met2)):
                                            if i == 0:
                                                label.append('B-metric')
                                            else:
                                                label.append('I-metric')

                                    if tag == 'd1':
                                        for i in range(len(dim1)):
                                            if i == 0:
                                                label.append('B-dimension')
                                            else:
                                                label.append('I-dimension')

                                    if tag == 'd2':
                                        for i in range(len(dim2)):
                                            if i == 0:
                                                label.append('B-dimension')
                                            else:
                                                label.append('I-dimension')

                                    if tag == 'f1':
                                        for i in range(len(fil1)):
                                            if i == 0:
                                                label.append('B-filter')
                                            else:
                                                label.append('I-filter')

                                    if tag == 'f2':
                                        for i in range(len(fil2)):
                                            if i == 0:
                                                label.append('B-filter')
                                            else:
                                                label.append('I-filter')

                                    if tag == 'pf1':
                                        for i in range(len(pfil1)):
                                            if i == 0:
                                                label.append('B-predefined_filter')
                                            else:
                                                label.append('I-predefined_filter')

                                    if tag == 'pf2':
                                        for i in range(len(pfil2)):
                                            if i == 0:
                                                label.append('B-predefined_filter')
                                            else:
                                                label.append('I-predefined_filter')
                                                
                                    if tag == 'sw':
                                        label.append('stop_word')

                                    if tag == 'fv1':
                                        for i in range(len(filv1.split())):
                                            if i == 0:
                                                label.append('B-filter_value')
                                            else:
                                                label.append('I-filter_value')
                                self.queries_with_tags.append(tuple(list(zip(rule[0].format(d1=' '.join(list(dim1)), 
                                                                                      m1=' '.join(list(met1)),
                                                                                      f1=' '.join(list(fil1)),
                                                                                      pf1=' '.join(list(pfil1)),
                                                                                      d2=' '.join(list(dim2)),
                                                                                      m2=' '.join(list(met2)),
                                                                                      f2=' '.join(list(fil2)),
                                                                                      pf2=' '.join(list(pfil2)),
                                                                                      fv1=filv1).split(),
                                                                                      label))))
        
                   
    def construct_name(self, word):
        return [word.split()[i] for i in sorted(list(set(rndint(0, len(word.split()), rndint(0, len(word.split()), 1)))))]
    
    def sent2features(self, sent):
        return [self.word2features(sent, i) for i in range(len(sent))]

    def sent2labels(self, sent):
        return [label for token, label in sent]

    def sent2tokens(self, sent):
        return [token for token, label in sent]  
    
    def add_features_to_words(self, words):
        res = np.zeros(4)
        names = ['metric', 'dimension', 'filter', 'predefined_filter']
        for ind, name in enumerate(names):
            try:
                c = self.dic_word_frequency[words[0]][name]
            except: 
                c = set()
            for word in words[1:]:
                try:
                    c = c.intersection(self.dic_word_frequency[word][name])
                except:
                    c = set()
            res[ind] = len(c)
        ans = np.argmax(res)
        res = np.zeros(4)
        res[ans] = 1
        return res

    def add_features_ratio(self, words):
        res = np.zeros(4)
        names = ['metric', 'dimension', 'filter', 'predefined_filter']
        for ind, name in enumerate(names):
            try:
                c = self.dic_word_frequency[words[0]][name]
            except: 
                c = set()
            for word in words[1:]:
                try:
                    c = c.intersection(self.dic_word_frequency[word][name])
                except:
                    c = set()
            if len(c) == 0:
                res[ind] = 0
            else:
                res[ind] = max([len(words) / len(self.dictionaries[name+"_dictionary"][pos[0]][1][pos[1]]) for pos in c])
        return res

    def word2features(self, sent, i):
        word = str(sent[i][0])
        # dict_features_non_stop = self.add_dictionary_features_to_word_non_stop(word)
        dict_features0 = self.add_features_to_words([word])
        features = list(itertools.chain.from_iterable([
            ['word.isdigit=%s' % word.isdigit(),
            'from_start=' + str(i),
            'from_end=' + str(len(sent) - i),
            'prop_start=' + str(i / len(sent)),
            'sent_len=' + str(len(sent)),
            'pos_tag=' + nltk.pos_tag([word])[0][1],],
            
            self.add_features_words_tag([word], '0:'),
            self.add_important_words_features_tag(word, '0:'),
            
        ]))

        if i > 0:
            word1 = str(sent[i - 1][0])
            dict_features = self.add_features_to_words([word1])
            dict_features1 = self.add_features_to_words([word, word1])
            features.extend(list(itertools.chain.from_iterable([
                ['-1:word.isdigit=%s' % word1.isdigit(),
                '-1:pos_tag=' + nltk.pos_tag([word1])[0][1],],
                
                self.add_features_words_tag([word], '-1:'),
                self.add_features_words_tag([word, word1], '-1.1:'),
                self.add_important_words_features_tag(word1, '-1:'),
                
                
            ])))
        else:
            features.append('BOS')

        if i > 1 and i < len(sent):
            word1 = str(sent[i - 2][0])
            dict_features1 = self.add_features_to_words([word, word1])
            features.extend(list(itertools.chain.from_iterable([
                ['-2:word.isdigit=%s' % word1.isdigit(),
                '-2:pos_tag=' + nltk.pos_tag([word1])[0][1],],
                
                self.add_features_words_tag([word, word1], '-2:'),
                self.add_important_words_features_tag(word1, '-2:'),
                
            ])))
        else:
            features.append('BOS')

        if i < len(sent) - 1:
            word1 = str(sent[i + 1][0])
            dict_features = self.add_features_to_words([word1])
            dict_features1 = self.add_features_to_words([word, word1])
            features.extend(list(itertools.chain.from_iterable([

                ['+1:word.isdigit=%s' % word1.isdigit(),
                '+1:pos_tag=' + nltk.pos_tag([word1])[0][1],],
                
                self.add_features_words_tag([word], '+1:'),
                self.add_features_words_tag([word, word1], '+1.1:'),
                self.add_important_words_features_tag(word1, '+1:'),
                
            ])))
        else:
            features.append('BOS')

        if i < len(sent) - 2:
            word1 = str(sent[i + 2][0])
            dict_features = self.add_features_to_words([word1])
            dict_features1 = self.add_features_to_words([word, word1])
            features.extend(list(itertools.chain.from_iterable([

                ['+2:word.isdigit=%s' % word1.isdigit(),
                '+2:pos_tag=' + nltk.pos_tag([word1])[0][1],],
                
                self.add_features_words_tag([word, word1], '+2:'),
                self.add_important_words_features_tag(word1, '+2:'),
                
                
            ])))
        else:
            features.append('BOS')

        if i > 0 and i < len(sent) - 1:
            word = str(sent[i][0])
            word1 = str(sent[i - 1][0])
            word2 = str(sent[i + 1][0])
            dict_features = self.add_features_to_words([word, word1, word2])
            #ratio_features = self.add_features_ratio([word, word1, word2])
            features.extend(list(itertools.chain.from_iterable([
                ['+1-1.0=' + str(dict_features[0]),
                '+1-1.1=' + str(dict_features[1]),
                '+1-1.2=' + str(dict_features[2]),
                '+1-1.3=' + str(dict_features[3]),
                ],
            ])))

        return features

    def sent2features(self, sent):
        return [self.word2features(sent, i) for i in range(len(sent))]

    def sent2labels(self, sent):
        return [label for token, label in sent]

    def sent2tokens(self, sent):
        return [token for token, label in sent]

    def train_test(self, split=0.5):
        self.X_train = [self.sent2features(s) for s in
                        tqdm(self.queries_with_tags[0:int(len(self.queries_with_tags) * split)])]
        self.y_train = [self.sent2labels(s) for s in self.queries_with_tags[0:int(len(self.queries_with_tags) * split)]]
        self.X_test = [self.sent2features(s) for s in
                       tqdm(self.queries_with_tags[int(len(self.queries_with_tags) * split) + 1:])]
        self.y_test = [self.sent2labels(s) for s in
                       self.queries_with_tags[int(len(self.queries_with_tags) * split) + 1:]]

    def add_features_to_query(self, query):
        
        q = [[token] for token in query.split()]
        self.total_numbers = np.array(
            [self.number_of_uniq_words_in_dictionary(self.dictionaries[d_name]) for d_name in self.dictionaries])
        return [self.sent2features(q)]
    
    def add_custom_query(self, words, tokens):
        DG.queries_with_tags.append(list(zip(words, tokens)))
        
    def add_important_words_features_tag(self, word, tag='0:'):
        list_of_words = ['by', 'for', 'where', 'with', 'contains', 'greater', 'less', 'equal']
        return [tag+w+'='+str(word==w) for w in list_of_words]
    
    def add_features_words_tag(self, words, tag='0:'):
        return [tag+str(ind)+'='+str(f) for ind, f in enumerate(self.add_features_to_words(words))]
    
    def add_features_ratio_tag(self, words, tag='0:'):
        return [tag+str(ind)+'='+str(f) for ind, f in enumerate(self.add_features_ratio(words))]

In [141]:
dictionaries = {'metric_dictionary': metric_dictionary,
                  'dimension_dictionary': dimension_dictionary,
                  'filter_dictionary': filter_dictionary, 
                  'predefined_filter_dictionary': predefined_filter_dictionary}

In [143]:
DG = DatasetGenerator(dictionaries=dictionaries)

In [145]:
DG.split_names_in_dictionaries()
DG.construct_dictionary_frequency({'metric': DG.dictionaries['metric_dictionary'],
                                   'dimension': DG.dictionaries['dimension_dictionary'],
                                   'filter': DG.dictionaries['filter_dictionary'],
                                   'predefined_filter': DG.dictionaries['predefined_filter_dictionary']})

In [157]:
query = "new users by source in germany"

In [158]:
res = DG.add_features_to_query(query)

In [159]:
list(map(len, res[0]))

[55, 78, 92, 92, 78, 55]